In [20]:
import tweepy
from dotenv import load_dotenv
import os
import pandas as pd
import re
load_dotenv()

True

In [6]:
tweepyKey = os.getenv("tweepyKey")
tweepyClient = tweepy.Client(tweepyKey)


In [7]:
def basic_count(query):
    count_df=pd.DataFrame(columns=['time', 'count'])
    response = tweepyClient.get_recent_tweets_count(query, granularity="hour")
    for count in response.data:
        data={'time':count['end'], 'count':count['tweet_count']}
        count_df=count_df.append(data, ignore_index=True)
    return count_df

In [27]:
# import nltk
# nltk.download('vader_lexicon')
# from textblob import TextBlob as tb
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# sid = SentimentIntensityAnalyzer()
import flair
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2022-05-31 20:37:50,936 loading file /Users/guruprasadmulay/.flair/models/sentiment-en-mix-distillbert_4.pt


In [46]:
# def clean_tweet(resp):
#     tweet=str(resp).lower()
#     tweet=re.sub(r"http\S+", '',tweet)
#     tweet=re.sub(r"www.\S+",'',tweet)
#     tweet=re.sub("[^A-Za-z0-9]"," ",tweet)
#     tweet=tweet.split()
#     tweet=' '.join(word for word in tweet)
#     return tweet

# TODO save tweets in file
from pathlib import Path
def tweets(start_time, end_time, iterations, query, useStoredData = False):
    # use dummy data reduce api calls
    if useStoredData:
        file_path = Path("Resources/dummyData.csv")
        return pd.read_csv(file_path)

    datelist = pd.date_range(start_time, end_time, periods=iterations).tolist()
    tweets=[]
    date=[]
    prob=[]
    sentim=[]
    counter = 0
    for time in datelist:
        if counter > 1:
            #Fetching Tweets
            response = tweepyClient.search_recent_tweets(
                query,
                max_results=100,
                start_time=start,
                end_time=time,
                tweet_fields=["created_at"])
    
            if type(response.data) != type(None):
                #Data Cleaning
                for resp in response.data:
                    # tweet = clean_tweet(resp)
                    tweet = str(resp)
                    if len(tweet) > 0:
                        # Sentiment analysis
                        sentence = flair.data.Sentence(tweet)
                        sentiment_model.predict(sentence)
                        tweets.append(tweet)
                        date.append(resp.created_at)
                        prob.append(sentence.labels[0].score)
                        sentim.append(sentence.labels[0].value)
        start=time
        counter = counter + 1

        # if counter == 3:
        #     break
        
    tweet_df=pd.DataFrame({'time':date, 'tweet':tweets, 'probability':prob, 'sentiment':sentim})
    tweet_df=tweet_df.sort_values(by='time', ascending=True).set_index('time')
    tweet_df.to_csv('Resources/dummyData.csv')
    return tweet_df

In [47]:
BTC=tweets('2022-05-29', '2022-05-30', 10, 'btc OR bitcoin OR crypto -is:retweet lang:en', useStoredData=True)
BTC

,time,tweet,probability,sentiment
0,2022-05-29 05:19:37+00:00,@1goonrich We're never getting rekt\nOne of th...,0.969354,POSITIVE
1,2022-05-29 05:19:37+00:00,RT @DMM_Bitcoin: ／\n毎日参加！その場で当たる！\n🎉 #フォローRTキャ...,0.952728,POSITIVE
2,2022-05-29 05:19:37+00:00,RT @airdropinspect: New airdrop: CloudChat (CC...,0.998047,POSITIVE
3,2022-05-29 05:19:37+00:00,RT @RecessionCoin: 24 HOURS UNTIL PRESALE!! Ma...,0.664618,NEGATIVE
4,2022-05-29 05:19:38+00:00,RT @DMM_Bitcoin: ／\n毎日参加！その場で当たる！\n🎉 #フォローRTキャ...,0.952728,POSITIVE
...,...,...,...,...
95,2022-05-29 05:19:59+00:00,@cryptoworld202 @omniaverse DApp features:\n\n...,0.973206,POSITIVE
96,2022-05-29 05:19:59+00:00,RT @dreamcity1991: #Shiba is the future of #ET...,0.997723,POSITIVE
97,2022-05-29 05:19:59+00:00,RT @LadyofCrypto1: ☝️ even if #Bitcoin turns a...,0.939273,NEGATIVE
98,2022-05-29 05:19:59+00:00,Meet the Steel Company that Wants to Become an...,0.997355,NEGATIVE
